# Gestor de investimento
---

In [1]:
import pandas as pd
import numpy as np
import os
import xlrd

# Profile 
from pandas_profiling import ProfileReport

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [93]:
# Função para pegar os dados de Papel (ações)
def get_stocks(df):
    stocks = []

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Papel' or row['Unnamed: 2'] == 'Ações':
            continue
        elif row['Unnamed: 2'] == 'Opções':
            break
        else:
            stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18', 
                                'Unnamed: 28', 'Unnamed: 35', 'Unnamed: 43', 
                                'Unnamed: 51', 'Unnamed: 61', 'Unnamed: 68', 
                                'Unnamed: 76', 'Unnamed: 83']].values)

    pd_stock = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetado', 'Qtd Dia', 
                                    'Qtd Garantia BOV', 'Qtd Garantia BMF', 
                                    'Qtd Estruturados', 'Liq Termo', 
                                    'Qtd Total', 'Cotacao', 'Financeiro'], data=stocks)

    return pd_stock


def get_picking(df):
    picking = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Ação':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Renda Fixa':
            break
        elif start == True:
            picking.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    pd_picking = pd.DataFrame(columns=['Papel', 'Qtd Provisionada', 
                                    'Tipo', 'Data Pagamento', 'Valor'], data=picking)
  
    return pd_picking


def get_fi(df):
    x = []
    start = False

    for index, row in df[~df.isnull()].iterrows():
        if row['Unnamed: 2'] == 'Fundos de Investimentos':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome Fundo':
            continue
        elif row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
            break
        elif (start == True) and ((type(row['Unnamed: 2']) == str)):
            if type(row['Unnamed: 2']) == str: 
                fi_name = np.array([row['Unnamed: 2']])
                data = df[df.index == index +1][['Unnamed: 13', 'Unnamed: 19', 
                                                    'Unnamed: 33', 'Unnamed: 40', 
                                                    'Unnamed: 48', 'Unnamed: 54', 
                                                    'Unnamed: 60', 'Unnamed: 70', 
                                                    'Unnamed: 81']]
                
                data.insert(0, 'Nome FI', row['Unnamed: 2'])
                # print(data.values[0])
                x.append(data.values[0])
                #display(data.head())
    
    pd_fi = pd.DataFrame(columns=['Nome', 'Data', 'Qtd Cotas', 
                                    'Valor Cota', 'Valor Bruto', 
                                    'IR', 'IOF', 'Valor Liquido', 
                                    'Aplicacao Pendente', 'Total Bruto'], data=x)
    return pd_fi


def get_fii(df):
    fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome':
            continue
        elif row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
            break
        elif start == True:
            fii.append(row[['Unnamed: 2', 'Unnamed: 14', 
                            'Unnamed: 26', 'Unnamed: 38', 
                            'Unnamed: 45', 'Unnamed: 55', 
                            'Unnamed: 74']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetada', 'Qtd Dia', 'Qtde Total', 
                                    'Ult Cotacao', 'Financeiro'], data=fii)
  
    return df_result


def get_picking_fii(df):
    p_fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Clubes de Investimentos':
            break
        elif start == True:
            p_fii.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Tipo', 
                                    'Qtd Provisionada', 'Dt Pagamento', 
                                    'Valor Provisionado'], data=p_fii)
  
    return df_result

In [94]:
files = os.listdir('../datasets/posicao/')

stocks = pd.DataFrame()
pickings = pd.DataFrame()
fis = pd.DataFrame()
fiis = pd.DataFrame()
picking_fii = pd.DataFrame()

for file_name in files:
    if file_name == '.DS_Store':
        continue
    
    wb = xlrd.open_workbook('../datasets/posicao/' + file_name, logfile=open(os.devnull, 'w'))
    df = pd.read_excel(wb)
    
    #df = pd.read_excel('../datasets/' + file_name)
    date_position = df[df['Unnamed: 56'].str.contains('Data de referência', na=False)]['Unnamed: 56']
    
    # position date
    date_position = pd.to_datetime(date_position.str.replace('Data de referência: ', ''), format='%d/%m/%Y')
    
    #month = int(date_position.dt.month.values)
    #year = int(date_position.dt.year.values)
    #period = str(year) + '/' + str(month)
    period = date_position.values[0]

    
    # get stocks
    df_stocks = get_stocks(df)
    #display(df_stocks.head())
    df_stocks['period'] = period
    stocks = stocks.append(df_stocks, ignore_index=True)

    df_pickings = get_picking(df)
    df_pickings['period'] = period
    pickings = pickings.append(df_pickings, ignore_index=True)

    df_fi = get_fi(df)
    df_fi['period'] = period
    fis = fis.append(df_fi, ignore_index=True)

    df_fii = get_fii(df)
    df_fii['period'] = period
    fiis = fiis.append(df_fii, ignore_index=True)

    df_picking_fii = get_picking_fii(df)
    df_picking_fii['period'] = period

    picking_fii = picking_fii.append(df_picking_fii, ignore_index=True)

# display(stocks.sort_values('period', ascending=False))
# display(fi)

In [95]:
picking_fii.sort_values(by='period').head()

,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
17,RNGO11,RENDIMENTO,49,14/02/2017,30.38,2017-01-31
16,RNGO11,RENDIMENTO,49,14/03/2017,29.40,2017-02-24
23,RNGO11,RENDIMENTO,49,17/04/2017,29.40,2017-03-31
21,RNGO11,RENDIMENTO,49,15/05/2017,24.99,2017-04-28
12,RNGO11,RENDIMENTO,49,14/06/2017,26.46,2017-05-31


In [96]:
fis['period'].sort_values().unique()

array(['2016-04-29T00:00:00.000000000', '2016-05-31T00:00:00.000000000',
       '2016-06-30T00:00:00.000000000', '2016-07-29T00:00:00.000000000',
       '2016-08-31T00:00:00.000000000', '2016-09-30T00:00:00.000000000',
       '2016-10-31T00:00:00.000000000', '2016-11-30T00:00:00.000000000',
       '2016-12-29T00:00:00.000000000', '2017-01-31T00:00:00.000000000',
       '2017-02-24T00:00:00.000000000', '2017-03-31T00:00:00.000000000',
       '2017-04-28T00:00:00.000000000', '2017-05-31T00:00:00.000000000',
       '2017-06-30T00:00:00.000000000', '2017-07-31T00:00:00.000000000',
       '2017-08-31T00:00:00.000000000', '2017-09-29T00:00:00.000000000',
       '2017-10-31T00:00:00.000000000', '2017-11-30T00:00:00.000000000',
       '2017-12-28T00:00:00.000000000', '2018-01-31T00:00:00.000000000',
       '2018-02-28T00:00:00.000000000', '2018-03-29T00:00:00.000000000',
       '2018-04-30T00:00:00.000000000', '2018-05-30T00:00:00.000000000',
       '2018-06-29T00:00:00.000000000', '2018-07-31

In [97]:
display(stocks.head())
display(pickings.head())
display(fis.head())
display(fiis.head())
display(picking_fii.head())

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period
0,JHSF3,1600,0,0,0,0,0,0,1600,7.01,"11,216.00",2020-02-28
1,MGLU3,800,0,0,0,0,0,0,800,50.52,"40,416.00",2020-02-28
2,MGLU3,800,0,0,0,0,0,0,800,37.04,"29,632.00",2019-09-30
3,BPAC11,200,0,0,0,0,0,0,200,48.84,"9,768.00",2020-05-29
4,ITSA4,0,500,0,0,0,0,0,500,8.84,"4,420.00",2020-05-29


,Papel,Qtd Provisionada,Tipo,Data Pagamento,Valor,period
0,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020-02-28
1,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020-02-28
2,MGLU3,800.00,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020-05-29
3,WEGE3,300.00,JUROS SOBRE CAPITAL PROPRIO,12/08/2020,9.07,2020-05-29
4,MGLU3,800.00,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020-05-29


,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
0,XP Long Short FIC FIM,2020-02-28,"6,902.02",2.91,"20,105.88",38.48,0,"20,067.40",0,"20,105.88",2020-02-28
1,Equitas Selection FIC FIA,2020-02-28,"3,018.52",7.23,"21,812.68",286.90,0,"21,525.78",0,"21,812.68",2020-02-28
2,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0,"18,874.07",0,"18,967.40",2020-02-28
3,Mauá Macro FIC FIM,2020-02-28,37.17,787.62,"29,278.24",186.19,0,"29,092.05",0,"29,278.24",2020-02-28
4,Bahia AM Maraú FIC de FIM,2020-02-28,"33,810.43",2.56,"86,466.95",67.77,0,"86,399.18",0,"86,466.95",2020-02-28


,Papel,Qtd Disponivel,Qtd Projetada,Qtd Dia,Qtde Total,Ult Cotacao,Financeiro,period
0,IBFF11,50,0,0,50.00,100.10,"5,005.00",2020-02-28
1,BCFF11,8,0,0,8.00,98.17,785.36,2020-02-28
2,CNES11,11,0,0,11.00,83.50,918.50,2020-02-28
3,OULG11B,50,0,0,50.00,105.00,"5,250.00",2020-02-28
4,RNGO11,49,0,0,49.00,96.61,"4,733.89",2020-02-28


,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
0,RNGO11,RENDIMENTO,49,13/03/2020,23.03,2020-02-28
1,RNGO11,RENDIMENTO,49,14/10/2019,24.01,2019-09-30
2,RNGO11,RENDIMENTO,49,14/08/2019,24.01,2019-07-31
3,RNGO11,RENDIMENTO,49,15/07/2019,26.95,2019-06-28
4,RNGO11,RENDIMENTO,49,15/05/2020,20.58,2020-04-30


# Analise de fundo imobiliario

In [98]:
report = ProfileReport(fis)

report.to_file(output_file='profile_teste.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 74.44it/s]


In [99]:
fis.sort_values(['Nome', 'period']).head()

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
139,AZ Quest Multi FIC FIM,2017-03-31,"1,516.42",9.95,"15,094.10",9.94,0,"15,034.29",0,"15,094.10",2017-03-31
128,AZ Quest Multi FIC FIM,2017-04-28,"1,516.42",10.00,"15,160.90",36.20,0,"15,124.70",0,"15,160.90",2017-04-28
88,AZ Quest Multi FIC FIM,2017-05-31,"1,516.42",9.73,"14,749.34",0.00,0,"14,749.34",0,"14,749.34",2017-05-31
92,AZ Quest Multi FIC FIM,2017-06-30,"1,516.42",9.80,"14,853.49",0.00,0,"14,853.49",0,"14,853.49",2017-06-30
114,AZ Quest Multi FIC FIM,2017-07-31,"1,516.42",10.15,"15,389.53",87.64,0,"15,301.89",0,"15,389.53",2017-07-31


In [100]:
fis[fis['Nome'] == 'AZ Quest Multi FIC FIM'].sort_values('period')

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
139,AZ Quest Multi FIC FIM,2017-03-31,"1,516.42",9.95,"15,094.10",9.94,0,"15,034.29",0,"15,094.10",2017-03-31
128,AZ Quest Multi FIC FIM,2017-04-28,"1,516.42",10.00,"15,160.90",36.20,0,"15,124.70",0,"15,160.90",2017-04-28
88,AZ Quest Multi FIC FIM,2017-05-31,"1,516.42",9.73,"14,749.34",0.00,0,"14,749.34",0,"14,749.34",2017-05-31
92,AZ Quest Multi FIC FIM,2017-06-30,"1,516.42",9.80,"14,853.49",0.00,0,"14,853.49",0,"14,853.49",2017-06-30
114,AZ Quest Multi FIC FIM,2017-07-31,"1,516.42",10.15,"15,389.53",87.64,0,"15,301.89",0,"15,389.53",2017-07-31
104,AZ Quest Multi FIC FIM,2017-08-31,"1,516.42",10.32,"15,644.93",145.10,0,"15,499.83",0,"15,644.93",2017-08-31
73,AZ Quest Multi FIC FIM,2017-09-29,"1,516.42",10.55,"16,004.66",200.93,0,"15,803.73",0,"16,004.66",2017-09-29
57,AZ Quest Multi FIC FIM,2017-10-31,"1,516.42",10.51,"15,934.88",186.97,0,"15,747.91",0,"15,934.88",2017-10-31
41,AZ Quest Multi FIC FIM,2017-11-30,"1,503.23",10.50,"15,784.72",46.16,0,"15,738.56",0,"15,784.72",2017-11-30
50,AZ Quest Multi FIC FIM,2017-12-29,"1,503.23",10.62,"15,969.51",83.11,0,"15,886.40",0,"15,969.51",2017-12-28


In [101]:
# Estou aqui!! Criar grafico comparando fundos por valor bruto

def plot_fis(df):

    for fi in df['Nome'].unique():
        fig = go.Figure()
        df_ = df[df['Nome'] == fi].sort_values('period')

        fig.add_trace(go.Scatter(x=df_['period'], y=df_['Qtd Cotas'] * df_['Valor Cota'], 
                                connectgaps=True,
                                mode='lines+markers',
                                name=fi, 
                                #text=round(df[mkt].astype(float)/1000000, 2).astype(str) + 'M', 
                                textposition="top center",
                                textfont={'color': 'rgb(100,100,100)', 'size':9}, 
                                # fill='tonexty', 
                                # df_offplatformByWeek['2020 10'].map('{:,.0f}'.format), 
                                marker=dict(size=3), 
                                opacity=.8))

        fig.update_layout(legend_orientation="h",
                        title=fi, 
                        height=300, 
                        width=600, 
                        legend=dict(font=dict(size=9)), 
                        template='plotly_white'                
        )

        fig.show()

plot_fis(fis)

# Analise aportes

In [102]:
def load_csv_extrato():
        df = pd.read_csv('../datasets/extrato/Extrato 200091 JAN 2010 a JUN 2020.csv', 
                            sep=';', encoding='iso-8859-1', decimal=',')
                
        df['Mov'] = pd.to_datetime(df['Mov'], format='%d/%m/%Y')
        df['Liq'] = pd.to_datetime(df['Liq'], format='%d/%m/%Y')
        df.rename(columns={'Hist__o': 'Descricao'}, inplace=True)

        return df

extract = load_csv_extrato()

In [103]:
fi_map = {
    'Equitas': 'Equitas Selection FIC FIA', 
    'Polo Norte': 'Polo Norte I FIC FIM', 
    'XP Macro': 'XP Macro FIM', 
    'Bahia AM Mara': 'Bahia AM Maraú FIC de FIM', 
    'QuestMult': 'AZ Quest Multi FIC FIM', 
    'Mau Macro FIC FIM': 'Mauá Macro FIC FIM', 
    'MiraeMacroStrategy': 'Mirae Asset Multimercado ', 
    'XP LONG SHORT': 'XP Long Short FIC FIM', 
    'XP GlobCredit': 'XP GlobCredit FICFIM', # Falta posicao deste periodo
    'Vot.FicFi CambialD': 'FICFIVot. CambDola',  # idem
    'FICFIVot. CambDola': 'FICFIVot. CambDola', 
    'Vot.FicFi CambialDol': 'FICFIVot. CambDola',
    'Inflacao Firf': 'BNP Inflacao Firf', # idem
    'Azul QuantitativoFIM': 'Azul Quantitativo',
    'QuantitativoFI': 'Azul Quantitativo', 
    'ABSOLUTO CONSUMO': 'ABSOLUTO CONSUMO', 
    'Hedge Fic Fim': 'Hedge Fic Fim',  
    'Legan Low Vol FIM': 'Legan Low Vol FIM', 
    'XP MULT-INV FIC FIA': 'XP MULT-INV FIC FIA'
}


def map_fi(x):
    group = "unknown"
    for key in fi_map:
        if key in x:
            group = fi_map[key]
            break
    return group


def get_fi_extract(df):
    df_aportes  = df[df['Descricao'].str.contains('TED APLICA')]
    df_aportes['Nome'] = df_aportes['Descricao'].apply(map_fi)

    # display(df_[df_['Nome'] == 'unknown'])

    df_resgates = df[df['Descricao'].str.contains('RESGATE')]\
                                .drop(df[df['Descricao'].str.contains('IRRF S/RESGATE FUNDOS|IRRF S/ RESGATE FUNDOS')].index)

    df_resgates_ir = df[df['Descricao'].str.contains('IRRF S/RESGATE FUNDOS|IRRF S/ RESGATE FUNDOS')]

    df_resgates['Nome'] = df_resgates['Descricao'].apply(map_fi)
    df_resgates_ir['Nome'] = df_resgates_ir['Descricao'].apply(map_fi)
    #df_['Nome'] = df_['Descricao'].map(lambda x: x.str.contains(fi_dict[0]))
    return df_aportes, df_resgates, df_resgates_ir

def extrato_fis(df):
    df_fi_aportes, df_fi_resgates, df_fi_ir = get_fi_extract(df)
    df_fi_aportes = df_fi_aportes.groupby('Nome')\
                                    .agg({'Valor': 'sum', 
                                        'Mov': 'max'})\
                                    .reset_index()\
                                    .rename(columns={'Valor': 'Vlr Aporte'})

    df_fi_resgates = df_fi_resgates.groupby('Nome')\
                                    .agg({'Valor': 'sum'})\
                                    .reset_index()\
                                    .rename(columns={'Valor': 'Vlr Resgate'})

    df_fi_ir = df_fi_ir.groupby('Nome')\
                                    .agg({'Valor': 'sum'})\
                                    .reset_index()\
                                    .rename(columns={'Valor': 'Vlr IR'})

    df_group = df_fi_aportes.merge(
                                df_fi_resgates, 
                                how='outer', 
                                left_on='Nome', 
                                right_on='Nome')\
                            .merge(df_fi_ir, 
                                    how='left', 
                                    left_on='Nome', 
                                    right_on='Nome'
                            ).fillna(0)

    #display(df_fi_aportes)
    #display(df_fi_resgates)

    df_group['Vlr Aporte'] = df_group['Vlr Aporte'].abs()
    df_group['Rendimento Resgatado'] = np.where(df_group['Vlr Resgate'] > 0, 
                                                df_group['Vlr Resgate'] - df_group['Vlr Aporte'],
                                                0)
    
    #df_group['Vlr Resgate'] - df_group['Vlr Aporte'] - df_group['Vlr IR'].abs()
    #df_group['Rendimento Resgatado'] = df_group['Rendimento Resgatado'].fillna(0)
    # df_group[df_group['Vlr Aporte'].isnull()]['Vlr Aporte'] = df_group[df_group['Vlr Aporte'].isnull()]['Vlr Resgate']

    return df_group

extrato_fi = extrato_fis(extract)


extrato_fi

,Nome,Vlr Aporte,Mov,Vlr Resgate,Vlr IR,Rendimento Resgatado
0,ABSOLUTO CONSUMO,"2,400.00",2013-05-28,"2,306.74",0.00,-93.26
1,AZ Quest Multi FIC FIM,"15,000.00",2017-03-17,"18,591.84",-37.00,"3,591.84"
2,Azul Quantitativo,"15,200.00",2013-08-23,"17,824.47",-522.50,"2,624.47"
3,BNP Inflacao Firf,"15,300.00",2015-02-26,"15,524.86",-179.53,224.86
4,Bahia AM Maraú FIC de FIM,"73,000.00",2017-10-10,0.00,0.00,0.00
5,Equitas Selection FIC FIA,"19,900.00",2019-09-12,0.00,0.00,0.00
6,FICFIVot. CambDola,"8,000.00",2015-09-10,"7,778.70",0.00,-221.30
7,Hedge Fic Fim,"5,000.00",2014-01-23,"5,144.33",-29.40,144.33
8,Legan Low Vol FIM,"10,000.00",2012-01-31,"10,333.81",-75.10,333.81
9,Mauá Macro FIC FIM,"23,500.00",2017-03-17,0.00,-111.82,0.00


In [104]:
display(extract.head())

,Liq,Mov,Descricao,Valor,Saldo
0,2020-06-04,2020-06-04,* PROV * DEBITO CBLC IRRF S/ RENDIMENTO DE BTC JHSF,-0.01,58.23
1,2020-06-04,2020-06-04,* PROV * Crdito Ref. Taxa de Remunerao BTC JHSF,0.06,58.24
2,2020-06-01,2020-05-28,OPERAÍES EM BOLSA PR 28/05/2020 NOTA N¼ 22035030,"-8,419.82",58.18
3,2020-05-28,2020-05-28,TED - RECEBIMENTO DE TED - SPB,"7,000.00","8,478.00"
4,2020-05-28,2020-05-28,RENDIMENTOS DE CLIENTES CNES11 S/ 11,1.81,"1,478.00"


In [105]:
extract.groupby('Descricao').agg({'Valor': 'sum'}).reset_index().sort_values(by='Valor').head()

,Descricao,Valor
177,DEPOSITO DE MARGEM DE GARANTIA,"-115,383.42"
594,TED TER BCO 237 AGE 2856 CTA 4563 2 - TED APLICAÌO FUNDOS Bahia AM Mara FIC FI,"-73,000.00"
155,CHAMADA DE MARGEM BM&F,"-57,617.05"
593,TED TER BCO 208 AGE 0001 CTA 183448 - TED APLICAÌO FUNDOS XP GlobCredit FICFIM,"-33,000.00"
586,TED TER BCO 017 AGE 0001 CTA 1310 2 - TED APLICAÌO FUNDOS Polo Norte I FIC FIM,"-30,000.00"


In [106]:
fis_ultima_posicao = fis.groupby(['Nome']).agg({'Data': 'max'}).reset_index()\
                            .merge(fis, 
                                    how='inner', 
                                    left_on=['Nome', 'Data'], 
                                    right_on=['Nome', 'Data']).reset_index()


df_rendimento = fis_ultima_posicao[['Nome', 
                                    'Valor Bruto']].merge(extrato_fi, 
                                        how='outer', 
                                        left_on='Nome', 
                                        right_on='Nome').reset_index()

df_rendimento['rendimento'] = np.where(df_rendimento['Vlr Resgate'] > 0, 
                                        df_rendimento['Vlr Resgate'], 
                                        df_rendimento['Valor Bruto']) - df_rendimento['Vlr Aporte'] 


df_rendimento['rendimento'].sum()

13364.520000000004

In [107]:
df_fi_stock, df_fi_draw = get_fi_extract(extract)

print('-- APORTE -- ')
# display(df_fi_stock.head())
display(df_fi_stock.groupby('Nome').agg({'Valor': 'sum'}).reset_index())
display(df_fi_stock['Valor'].sum())


print('-- RESGATE -- ')
# display(df_fi_draw)
display(df_fi_draw.groupby('Nome').agg({'Valor': 'sum'}).reset_index())
display(df_fi_draw['Valor'].sum())


ValueError: too many values to unpack (expected 2)

In [108]:
extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168 entries, 0 to 1167
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Liq        1168 non-null   datetime64[ns]
 1   Mov        1168 non-null   datetime64[ns]
 2   Descricao  1168 non-null   object        
 3   Valor      1168 non-null   float64       
 4   Saldo      1168 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 45.8+ KB


In [109]:
extract[extract['Descricao'].str.contains('QuestMult')].sort_values('Mov')




,Liq,Mov,Descricao,Valor,Saldo
196,2017-03-17,2017-03-17,TED TER BCO 017 AGE 0001 CTA 1518 0 - TED APLICAÌO FUNDOS QuestMult FI CotasFI,"-15,000.00",80.67
23,2020-03-16,2020-03-16,IRRF S/RESGATE FUNDOS QuestMult FI CotasFI,-37.00,"24,064.50"
24,2020-03-16,2020-03-16,RESGATE QuestMult FI CotasFI,"18,591.84","24,101.50"


In [110]:
display(extract.groupby('Descricao').agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).head())
display(extract.groupby('Descricao').agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).tail(5)) # Aportes para fundos

,Descricao,Valor
582,TED - RECEBIMENTO DE TED - SPB,"253,400.00"
180,DEVOLUÌO DE MARGEM BM&F,"61,053.65"
515,RETIRADA DE MARGEM DE GARANTIA,"58,543.73"
179,DEVOLUÌO DE MARGEM - NÌO NETADA,"47,568.07"
512,RESGATE XP GlobCredit FICFIM,"31,029.08"


,Descricao,Valor
586,TED TER BCO 017 AGE 0001 CTA 1310 2 - TED APLICAÌO FUNDOS Polo Norte I FIC FIM,"-30,000.00"
593,TED TER BCO 208 AGE 0001 CTA 183448 - TED APLICAÌO FUNDOS XP GlobCredit FICFIM,"-33,000.00"
155,CHAMADA DE MARGEM BM&F,"-57,617.05"
594,TED TER BCO 237 AGE 2856 CTA 4563 2 - TED APLICAÌO FUNDOS Bahia AM Mara FIC FI,"-73,000.00"
177,DEPOSITO DE MARGEM DE GARANTIA,"-115,383.42"


In [111]:
display(extract.groupby(['Descricao', 'Mov']).agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False)

SyntaxError: unexpected EOF while parsing (<ipython-input-111-98c495828340>, line 1)